In [ ]:
import sys
sys.path.append('../../')

from SynTemp.SynITS.its_extraction import ITSExtraction
from SynTemp.SynUtils.utils import load_database, save_database

toy_sample = [{'balanced': True,
  'R-id': 'USPTO_50K_31',
  'reactions': 'C=C1C(=C)C2OC1C(=C)C2=C.C=CC(C)=O>>C=C1C(=C)C2OC1C1=C2CC(C(C)=O)CC1',
  'local_mapper': '[CH2:4]=[C:3]1[C:2](=[CH2:1])[CH:7]2[O:6][CH:5]1[C:9](=[CH2:10])[C:8]2=[CH2:16].[CH2:15]=[CH:11][C:12]([CH3:13])=[O:14]>>[CH2:1]=[C:2]1[C:3](=[CH2:4])[CH:5]2[O:6][CH:7]1[C:8]1=[C:9]2[CH2:10][CH:11]([C:12]([CH3:13])=[O:14])[CH2:15][CH2:16]1',
  'rxn_mapper': '[CH2:1]=[C:2]1[C:3](=[CH2:4])[CH:5]2[O:6][CH:7]1[C:8](=[CH2:16])[C:9]2=[CH2:15].[CH2:10]=[CH:11][C:12]([CH3:13])=[O:14]>>[CH2:1]=[C:2]1[C:3](=[CH2:4])[CH:5]2[O:6][CH:7]1[C:8]1=[C:9]2[CH2:10][CH:11]([C:12]([CH3:13])=[O:14])[CH2:15][CH2:16]1',
  'graphormer': '[CH2:1]=[C:2]1[C:3](=[CH2:4])[CH:5]2[C:10](=[CH2:11])[C:8](=[CH2:9])[CH:7]1[O:6]2.[CH3:15][C:14](=[O:16])[CH:13]=[CH2:12]>>[CH3:12][C:15]([CH:14]1[CH2:13][CH2:11][C:10]2=[C:8]([CH:7]3[O:6][CH:5]2[C:3](=[CH2:4])[C:2]3=[CH2:1])[CH2:9]1)=[O:16]',
  'Valid': True}]

# **1. ITS and Templates Extraction**

In [ ]:
from SynTemp.SynVis.chemical_reaction_visualizer import ChemicalReactionVisualizer
reaction_vis = ChemicalReactionVisualizer()
reaction_vis.visualize_reaction(toy_sample[0]['rxn_mapper'])

In [ ]:
reaction_vis.visualize_reaction(toy_sample[0]['local_mapper'])

In [ ]:
reaction_vis.visualize_reaction(toy_sample[0]['graphormer'], show_atom_map=True)

In [ ]:
from SynTemp.SynVis.its_visualizer import ITSVisualizer
from IPython.display import Image
its_vis = ITSVisualizer(toy_sample[0]['rxn_mapper'])
display(Image(its_vis.draw_product_with_modified_bonds()))

In [ ]:
its_vis = ITSVisualizer(toy_sample[0]['local_mapper'])
display(Image(its_vis.draw_product_with_modified_bonds()))

In [ ]:
its_vis = ITSVisualizer(toy_sample[0]['graphormer'])
display(Image(its_vis.draw_product_with_modified_bonds()))

In [ ]:
# Extract rules and graph rules based on Imaginary Transition State (ITS)
mapper_name = ['rxn_mapper', 'graphormer', 'local_mapper']
its_graph, its_graph_wrong = ITSExtraction.parallel_process_smiles(toy_sample, mapper_name, threshold=2, n_jobs=5, verbose=1, export_full=False, check_method='RC')

In [ ]:
its_graph

In [ ]:
its_graph_wrong[0]

In [ ]:
from SynTemp.SynVis.chemical_graph_visualizer import ChemicalGraphVisualizer

vis = ChemicalGraphVisualizer(seed=42)
vis.visualize_all(its_graph_wrong[0]['graphormer'])

In [ ]:
its_graph

In [ ]:
import sys
sys.path.append('../../')
from SynTemp.SynAAM.aam_postprocess import AMMPostprocessor
from SynTemp.SynITS.its_extraction import ITSExtraction
from SynTemp.SynUtils.utils import load_database, save_database

data = load_database(f'../../Data/uspto/uspto_balance_aam_reactions.json.gz')
mapper_name = ['rxn_mapper', 'graphormer', 'local_mapper']

# make sure reactions all have atom mapping
check_valid = AMMPostprocessor.parallel_postprocess(data, mapper_name, threshold=3, n_jobs=5, verbose=1)
valid_data = [reaction for reaction in check_valid if reaction.get('Valid')]

In [ ]:
# Extract rules and graph rules based on Imaginary Transition State (ITS)
its_graph, its_graph_wrong = ITSExtraction.parallel_process_smiles(valid_data, mapper_name, threshold=2, n_jobs=5, verbose=1, export_full=False, check_method='RC')

len(its_graph)